Information about LZKN archives in Contra Hard Corps ROMs
---
Script open all LZKN archives in Contra Hard Corps ROM and calculate their sizes.
It uses lzkn1.dll lib https://github.com/spiiin/lzkn to calculate sizes or archives and file lzkn1_list.txt to read addresses of all arrays.
(Original addresses was collected with lua-script, that dumps addressess while decompressing data in emulator)

In [1]:
from ctypes import *
from IPython.display import display
from ipy_table import *

ROM_NAME = "Contra - Hard Corps (U) [!].gen"
TXT_NAME = "lzkn1_list.txt"

CHUNK_SIZE = 8192
MAX_OUT_SIZE = 10**6

lzkn1 = cdll.LoadLibrary("lzkn1.dll")

with open(ROM_NAME, "rb") as f:
    data = f.read()
data = map(ord, data)

with open(TXT_NAME, "rt") as f:
    lines = f.readlines()
lines = list(x.strip() for x in lines)

toTable =[]
toTable.append(["LZKN address", "Decompressed size", "Original ROM compressor", "LZKN compressor [lab313]", "delta"])
    
for line in lines:
    curAddres = int(line, 16)
    chunkData = data[curAddres:curAddres+CHUNK_SIZE]
    compressedSize = c_int()
    c_data = (c_ubyte * len(chunkData)) (*chunkData)
    c_data_p = POINTER(c_ubyte) (c_data)
    c_uncompressedData = create_string_buffer(MAX_OUT_SIZE)
    decompressedSize = lzkn1.decompress(c_data_p, byref(c_uncompressedData), byref(compressedSize))
    
    #  with open("map_" + hex(curAddres), "wb") as f:
    #      f.write(c_uncompressedData[:decompressedSize])
    
    c_recompressedData = create_string_buffer(decompressedSize)
    recompressedSize = lzkn1.compress(byref(c_uncompressedData), byref(c_recompressedData), decompressedSize)
    
    #  with open("lzkn_" + hex(curAddres), "wb") as f:
    #      f.write(c_recompressedData[:recompressedSize])
    
    #print "Lzkn address: %8s"%hex(curAddres),
    #print " Decompressed size: %4d"%decompressedSize,
    #print " Compressed size: %4d"%compressedSize.value,
    #print " Recompressed size: %4d"%recompressedSize,
    #print " Difference with original: %4d"%(compressedSize.value - recompressedSize)

    toTable.append([hex(curAddres), decompressedSize, compressedSize.value, recompressedSize, (compressedSize.value - recompressedSize)])
    
t = make_table(toTable)
t.apply_theme("basic")
display(t)
    



LZKN address,Decompressed size,Original ROM compressor,LZKN compressor [lab313],delta
0x9ed6,1536,893,871,22
0x12a000,8192,4515,4394,121
0x12b1a4,8192,3293,3199,94
0x12c5a0,8192,2384,2324,60
0x12cef0,4096,1915,1862,53
0x12d66c,1024,348,346,2
0x12d7c8,8192,4888,4752,136
0x12eae0,8192,5920,5782,138
0x130200,8192,4743,4637,106
0x131488,4096,2334,2276,58
